In [15]:
#THIS IS THE FINAL SCRIPT
import DataStructures
@time begin
    
struct Info
    spectrum
end

mutable struct Cyclic
    p::Info
    geneticCode
end
function expand(c::Cyclic, peptides)
    expand = DataStructures.DefaultDict(0)
        for (pep, mass) in peptides
            for aa in keys(c.geneticCode)
                expand[string(pep, aa)] = mass + c.geneticCode[aa]
            end
        end
    return expand
end
function cyclospectrum(c::Cyclic, peptide)  
    prefixMass = [0]
    for i in 1:length(peptide)
        for (k,v) in c.geneticCode
            if k === peptide[i]
                push!(prefixMass, prefixMass[i] + v)
            end
        end 
    end
    peptideMass = prefixMass[end]
    cyclicSpectrum = [0]
    for i in range(start = 1 , stop = (length(peptide)), step = 1)
        for j in range(start = (i+1), stop = (length(peptide)+1) , step = 1)
            push!(cyclicSpectrum, prefixMass[j] - prefixMass[i])
            if i > 1 && j <= length(peptide)
                push!(cyclicSpectrum, peptideMass - (prefixMass[j] - prefixMass[i]))
            end
        end
    end
    cyclicSpectrum = sort!(cyclicSpectrum)
    return cyclicSpectrum
               

end
function cycloPeptide(c::Cyclic)
    aminoAcid = Dict('G'=> 57, 'A'=> 71, 'S'=> 87, 'P'=> 97, 'V'=> 99, 'T'=> 101, 'C'=> 103, 'I'=> 113, 'N'=> 114,
                     'D'=> 115, 'K'=> 128, 'E'=> 129, 'M'=> 131, 'H'=> 137, 'F'=> 147, 'R'=> 156, 'Y'=> 163, 'W'=> 186)
    peptides = Dict{String, Integer}("" => 0)
    parentMass = findmax(c.p.spectrum)[1]
    for i in range(start = 1, stop = length(c.p.spectrum), step = 1)
        for (k,v) in aminoAcid
            if c.p.spectrum[i] == v
                c.geneticCode[k] = v
            end
        end
    end
    output = String[]
    while length(peptides) != 0
        peptides = expand(c, peptides)
        temp1 = deepcopy(peptides)
        for (peptide, mass) in temp1
            if mass == parentMass
                if cyclospectrum(c,peptide) == c.p.spectrum
                    push!(output, peptide)
                end
                delete!(peptides, peptide)
            elseif mass ∉ c.p.spectrum
                delete!(peptides, peptide)
            else
            end
        end
    end
        
    outputPeptide = []
    tempOut = []
    for pep in output
        temp = ""
        i = []
        for p in pep
            push!(i, aminoAcid[p])            
        end
        push!(tempOut, i) 
    end
    tempOut = sort(tempOut)
    outputPeptide = []    
    for pep in tempOut
        temp = ""
        for p in pep
            s = string(p) * "-"
            temp = temp * s  
        end
        push!(outputPeptide, temp[1:end-1])
        #println(temp[1:end-1])
    end
   println(length(outputPeptide))   
end

function main()
     open("rosalind_ba4e_762_6_dataset.txt") do f
      spectrum = [] 
          # read till end of file
      while ! eof(f) 
         # read a new / next line for every iteration          
         s = readline(f)           
         input = split(s)         
         for i in input
            push!(spectrum, parse(Int64, i))
         end      
            
      end
      #println(spectrum)
      @time begin
      p = Info(spectrum)
      geneticCode = DataStructures.DefaultDict(0)
      c =Cyclic(p, geneticCode)
      cycloPeptide(c)
      end
    end
end
    
    

main()
end

22
 36.166471 seconds (21.87 M allocations: 463.094 MiB, 76.57% gc time, 4.88% compilation time)
128.403642 seconds (22.13 M allocations: 477.331 MiB, 52.67% gc time, 69.94% compilation time)
